# Facial Recognition Demo

### Set ENV variables

In [1]:
%run ./components/util/set_env.ipynb

### Setup project

In [2]:
import os
from os import path, getenv
import nuclio
from mlrun import new_project, run_local, NewTask, mlconf, import_function, mount_v3io

project_name = os.getenv("PROJECT")
project_path = path.abspath('project')
project = new_project(name=project_name, context=project_path)
artifact_path = path.abspath('artifacts')
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Project path: {project_path}\nProject name: {project_name}')
print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Project path: /User/iguazioCV/project
Project name: facial-recognition-demo
Artifacts path: /User/iguazioCV/artifacts
MLRun DB path: http://mlrun-api:8080


### Build Docker Image

In [3]:
# Run once to create Docker image
# %run components/notebooks/CreateDockerImage.ipynb

### Build Pipeline Functions

Run the following notebooks in `components/notebooks` to populate `components/yaml`:
- `CreateStreamsTables.ipynb`
- `DeployFacialRecognition.ipynb`
- `DeployImageRetrieval.ipynb`
- `CreateAPIGateway.ipynb`
- `CreateGrafanaDashboard.ipynb`

*Note: Be sure to update Docker image in notebooks if using a different image than the one created previously.*

### Set Pipeline Functions

In [4]:
project.set_function('../components/yaml/create-streams-tables.yaml', 'create-streams-tables')
project.set_function('../components/yaml/deploy-facial-recognition.yaml', 'deploy-facial-recognition')
project.set_function('../components/yaml/deploy-image-retrieval.yaml', 'deploy-image-retrieval')
project.set_function('../components/yaml/create-api-gateway.yaml', 'create-api-gateway')
project.set_function('../components/yaml/create-grafana-dashboard.yaml', 'create-grafana-dashboard')

### Pipeline

In [5]:
%%writefile {path.join(project_path, 'workflow.py')}

import os
from kfp import dsl
from mlrun import mount_v3io

funcs = {}

# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name="Facial Recognition Demo",
    description="Real-time facial recognition using Iguazio Data Science Platform, Nuclio, and MLRun"
)
def kfpipeline():
    # Note: Using env/params to order components as everything has been defined in environment variables.
    # "env" is the input for nuclio functions which output a function "endpoint"
    # "params" is the input for jobs which output a "run_id"

    # Create streams and tables
    create_streams_tables = funcs['create-streams-tables'].as_step(outputs=['tagged_video_stream_url', 
                                                                            'raw_video_stream_url'])
    
    # Deploy facial recognition
    facial_recognition = funcs['deploy-facial-recognition'].deploy_step(env={'RUN_ORDER' : create_streams_tables.outputs['run_id'],
                                                                             'TAGGED_VIDEO_STREAM_URL' : create_streams_tables.outputs['tagged_video_stream_url']})

    # Deploy image retrieval
    image_retrieval = funcs['deploy-image-retrieval'].deploy_step(env={'RUN_ORDER' : facial_recognition.outputs['endpoint']})

    # Create API gateway
    api_gateway = funcs['create-api-gateway'].as_step(params={'RUN_ORDER' : image_retrieval.outputs['endpoint']})
    
    # Create Grafana dashboard
    api_gateway = funcs['create-grafana-dashboard'].as_step(params={'RUN_ORDER' : api_gateway.outputs['run_id']})

Overwriting /User/iguazioCV/project/workflow.py


### Save pipeline

In [6]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [7]:
project.save()

### Run pipeline

In [8]:
run_id = project.run(
    'main',
    arguments={}, 
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}')), 
    dirty=True)

> 2020-10-28 20:50:09,943 [info] using in-cluster config.


> 2020-10-28 20:50:10,353 [info] Pipeline run id=e0c02073-c0f0-4a17-aab9-49b014e43f30, check UI or DB for progress


### Clean Up Resources

In [9]:
# %run ./components/util/cleanup.ipynb

Deleting API Gateway 'camera-img'...
Deleting Function 'facial-recognition-demo-latest-image-retrieve'...
Deleting Function 'facial-recognition-demo-deploy-facial-recognition'...
Deleting Stream 'videostream'...
Deleting Stream 'taggedvideos'...
Deleting Table 'camera_list'...
Done.
